# AOAI Batch BYO storage via SAS tokens Private preview - Python SDK

Get started with Azure OpenAI Service Batch Inferencing with Bring Your Own Storage using this notebook.

You will learn how to use the Azure OpenAI Service python API to generate chat completions asynchronously with batch inference using your own hosted Blob Storage account.

## Create GlobalBatch deployment

If you haven't already, [create an Azure OpenAI Service resource](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/create-resource?pivots=web-portal).

Then create a deployment. Be sure to create a deployment with "Global Batch" deployment type.

## Load account configuration

Edit `config.json` to add you [Azure OpenAI Service API key](https://learn.microsoft.com/en-us/answers/questions/1193991/how-to-get-the-value-of-openai-api-key), Azure OpenAI account, and deployment, as well as your [Azure Blob Storage](https://learn.microsoft.com/en-us/azure/storage/blobs/storage-blobs-introduction) account name, input container, and output container. 

```json
{
    "api_key": "<your-subscription-key>",
    "aoai_account": "<your-aoai-account>",
    "aoai_resource_group": "<your-aoai_resource_group>",
    "aoai_subscription_id": "<your-aoai-subscription-id>",
    "aoai_deployment": "<your-aoai-deployment>",
    "azure_blob_account_name": "<your-azure-blob-account-name>",
    "input_container": "<your-input-container>",
    "output_container": "<your-output-container>"
}
```

In [ ]:
!pip install -r requirements.txt

In [ ]:
!az login

In [ ]:
import json 
import requests
import time
from datetime import datetime

# Load the config file
with open("config.json") as f:
    config = json.load(f)

aoai_account_name = config["aoai_account"]
aoai_resource_group = config["aoai_resource_group"]
aoai_subscription_id = config["aoai_subscription_id"]
apim_subscription_id = config["apim_subscription_id"]
deployment = config["aoai_deployment"]
api_key = config["api_key"]
blob_account_name = config["azure_blob_account_name"]
input_container = config["input_container"]
output_container = config["output_container"]
input_file_name = config["input_file_name"]
output_folder_name = config["output_folder_name"]

print("Account: ", aoai_account_name)
print("Resource group: ", aoai_resource_group)
print("Subscription ID: ", aoai_subscription_id)
print("APIM subscription ID: ", apim_subscription_id)
print("Deployment: ", deployment)
print("API Key: ", api_key)
print("Blob account name: ", blob_account_name)
print("Input container: ", input_container)
print("Input file name: ", input_file_name)
print("Output container: ", output_container)
print("Output folder name: ", output_folder_name)

In [55]:
import os, uuid
from datetime import datetime, timedelta
from azure.identity import DefaultAzureCredential
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

account_url = f"https://{blob_account_name}.blob.core.windows.net"
default_credential = DefaultAzureCredential()

blob_service_client = BlobServiceClient(account_url, credential=default_credential)
user_delegation_key = blob_service_client.get_user_delegation_key(datetime.utcnow(), datetime.utcnow() + timedelta(hours=2))

In [56]:
def get_or_create_container_client(blob_service_client, container_name):
    client = blob_service_client.get_container_client(container_name)
    if not client.exists():
        client.create_container()
    return client

input_client = get_or_create_container_client(blob_service_client, input_container)
output_client = get_or_create_container_client(blob_service_client, output_container)

In [63]:
from azure.storage.blob import generate_container_sas, ContainerSasPermissions

def generate_sas_token(blob_service_client, container_name, container_sas_permission, expiry):
    return generate_container_sas(
        account_name=blob_service_client.account_name,
        container_name=container_name,
        user_delegation_key=user_delegation_key,
        permission=container_sas_permission,
        expiry=expiry
    )

expiry = datetime.utcnow() + timedelta(hours=50)  # Token valid for 2 days + some extra

input_sas_perms = ContainerSasPermissions(read=True)
input_sas = generate_sas_token(blob_service_client, input_container, input_sas_perms, expiry)

output_sas_perms = ContainerSasPermissions(read=True, write=True, list=True)
output_sas = generate_sas_token(blob_service_client, output_container, output_sas_perms, expiry)

In [ ]:
print(input_sas)

In [59]:
with open("input.jsonl") as f:
    text = f.read().replace("{{modelName}}", deployment)
    input_client.upload_blob(input_file_name, text, overwrite=True)

In [60]:
import uuid

url = f"https://{aoai_account_name}.openai.azure.com/openai/batches?api-version=2024-07-01-preview"
aoai_resource_id = f"/subscriptions/{aoai_subscription_id}/resourceGroups/{aoai_resource_group}/providers/Microsoft.CognitiveServices/accounts/{aoai_account_name}"
request_id = str(uuid.uuid4())
headers = {
  'api-key': api_key,
  'apim-subscription-id': apim_subscription_id,
  'apim-request-id': request_id,
  'Azure-Resource-Id': aoai_resource_id,
  'Content-Type': 'application/json'
}

input_container_sas_url = f"{input_client.url}?{input_sas}"
output_container_sas_url = f"{output_client.url}?{output_sas}"

payload = json.dumps({
  "input_file_reference": {
    "container_sas_url": input_container_sas_url,
     "relative_file_path": input_file_name
  },
  "output_folder_reference": {
    "container_sas_url": output_container_sas_url,
    "relative_folder_path": output_folder_name
  }
})

response  = requests.request("POST", url, headers=headers, data=payload)
print(response.text)

In [ ]:
batch_id = response.json()['id']
print(batch_id)

def get_batch(batch_id):
    url = f"https://{aoai_account_name}.openai.azure.com/openai/batches/{batch_id}?api-version=2024-07-01-preview"
    return requests.request("GET", url, headers=headers)

terminal_statuses = ["Completed", "Failed", "Canceled"]
while True:
    response = get_batch(batch_id=batch_id)
    status = response.json()['status']
    print(f"{datetime.now()} Batch Id: {batch_id}, Status: {status}")
    if status in terminal_statuses:
        break

    time.sleep(15)